In [57]:
import tensorflow as tf
import numpy as np
# import librosa
# import librosa.display
import matplotlib.pyplot as plt

filename = 'filtered_data_keyboard.tfrecord'
dataset = tf.data.TFRecordDataset(filename)

# Suppose you have 3 workers in total
# with tf.Session() as sess:
#     for i in range(100):
#         value = sess.run(res)
#         example = tf.train.Example()
#         example.ParseFromString(value)
#         f = example.features.feature
#         print(f['instrument_family'].int64_list.value[0])

In [58]:
epoch = 10
dataset = dataset.repeat(epoch)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()

num_batch = 0
j = 0
audio_arr = []
instrumentFamily_arr = []
while True:
    value = sess.run(next_element)
    example = tf.train.Example()
    example.ParseFromString(value)
    f = example.features.feature
    audio = np.array(f['audio'].float_list.value)
    audio_arr.append(audio)
    instrumentFamily = np.int32(f['instrument_family'].int64_list.value[0])
    instrumentFamily_arr.append(instrumentFamily)
    j += 1
    if j>10:
        break
print("Done")
audio_arr = np.asarray(audio_arr)
instrumentFamily_arr = np.asarray(instrumentFamily_arr)

Done


In [88]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )

    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )

    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
          loss=loss,
          global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [101]:
def main(unused_argv):
    # Load training and eval data
#     mnist = tf.contrib.learn.datasets.load_dataset("mnist")
#     train_data = mnist.train.images # Returns np.array
    train_data = audio_arr
    print(type(train_data))
    print(type(train_data[0]))
#     train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    train_labels = instrumentFamily_arr
    print(train_labels)
#     eval_data = mnist.test.images # Returns np.array
#     eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
#     mnist_classifier = tf.estimator.Estimator(
#         model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")
#     tensors_to_log = {"probabilities": "softmax_tensor"}
#     logging_hook = tf.train.LoggingTensorHook(
#       tensors=tensors_to_log, every_n_iter=50)
#     # Train the model
#     train_input_fn = tf.estimator.inputs.numpy_input_fn(
#         x={"x": train_data},
#         y=train_labels,
#         batch_size=100,
#         num_epochs=None,
#         shuffle=True)
#     mnist_classifier.train(
#         input_fn=train_input_fn,
#         steps=20000,
#         hooks=[logging_hook])
#     eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#       x={"x": eval_data},
#       y=eval_labels,
#       num_epochs=1,
#       shuffle=False)
#     eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)

In [102]:
if __name__ == "__main__":
    tf.app.run()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[4 4 4 4 4 4 4 4 4 4 4]


SystemExit: 

C:\Users\Brian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
